# Исследование надежности заемщиков


Во второй части проекта вы выполните шаги 3 и 4. Их вручную проверит ревьюер.
Чтобы вам не пришлось писать код заново для шагов 1 и 2, мы добавили авторские решения в ячейки с кодом. 



## Откройте таблицу и изучите общую информацию о данных

**Задание 1. Импортируйте библиотеку pandas. Считайте данные из csv-файла в датафрейм и сохраните в переменную `data`. Путь к файлу:**

`/datasets/data.csv`

In [9]:
import pandas as pd

try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

**Задание 2. Выведите первые 20 строчек датафрейма `data` на экран.**

In [3]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Задание 3. Выведите основную информацию о датафрейме с помощью метода `info()`.**

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

**Задание 4. Выведите количество пропущенных значений для каждого столбца. Используйте комбинацию двух методов.**

In [10]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

**Задание 5. В двух столбцах есть пропущенные значения. Один из них — `days_employed`. Пропуски в этом столбце вы обработаете на следующем этапе. Другой столбец с пропущенными значениями — `total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца `income_type`. Например, у человека с типом занятости `сотрудник` пропуск в столбце `total_income` должен быть заполнен медианным доходом среди всех записей с тем же типом.**

In [11]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

**Задание 6. В данных могут встречаться артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. таким артефактом будет отрицательное количество дней трудового стажа в столбце `days_employed`. Для реальных данных это нормально. Обработайте значения в этом столбце: замените все отрицательные значения положительными с помощью метода `abs()`.**

In [12]:
data['days_employed'] = data['days_employed'].abs()

**Задание 7. Для каждого типа занятости выведите медианное значение трудового стажа `days_employed` в днях.**

In [13]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставьте их как есть. Тем более этот столбец не понадобится вам для исследования.

**Задание 8. Выведите перечень уникальных значений столбца `children`.**

In [14]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

**Задание 9. В столбце `children` есть два аномальных значения. Удалите строки, в которых встречаются такие аномальные значения из датафрейма `data`.**

In [15]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

**Задание 10. Ещё раз выведите перечень уникальных значений столбца `children`, чтобы убедиться, что артефакты удалены.**

In [16]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

### Удаление пропусков (продолжение)

**Задание 11. Заполните пропуски в столбце `days_employed` медианными значениями по каждого типа занятости `income_type`.**

In [17]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

**Задание 12. Убедитесь, что все пропуски заполнены. Проверьте себя и ещё раз выведите количество пропущенных значений для каждого столбца с помощью двух методов.**

In [18]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

**Задание 13. Замените вещественный тип данных в столбце `total_income` на целочисленный с помощью метода `astype()`.**

In [19]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

**Задание 14. Обработайте неявные дубликаты в столбце `education`. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведите их к нижнему регистру.**

In [20]:
data['education'] = data['education'].str.lower()

**Задание 15. Выведите на экран количество строк-дубликатов в данных. Если такие строки присутствуют, удалите их.**

In [21]:
data.duplicated().sum()

71

In [22]:
data = data.drop_duplicates()

### Категоризация данных

**Задание 16. На основании диапазонов, указанных ниже, создайте в датафрейме `data` столбец `total_income_category` с категориями:**

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.


**Например, кредитополучателю с доходом 25000 нужно назначить категорию `'E'`, а клиенту, получающему 235000, — `'B'`. Используйте собственную функцию с именем `categorize_income()` и метод `apply()`.**

In [23]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [24]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

**Задание 17. Выведите на экран перечень уникальных целей взятия кредита из столбца `purpose`.**

In [19]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

**Задание 18. Создайте функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:**

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

**Например, если в столбце `purpose` находится подстрока `'на покупку автомобиля'`, то в столбце `purpose_category` должна появиться строка `'операции с автомобилем'`.**

**Используйте собственную функцию с именем `categorize_purpose()` и метод `apply()`. Изучите данные в столбце `purpose` и определите, какие подстроки помогут вам правильно определить категорию.**

In [26]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [27]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

### Шаг 3. Исследуйте данные и ответьте на вопросы

#### 3.1 Есть ли зависимость между количеством детей и возвратом кредита в срок?

Для начала создадим таблицу, куда включим соотношение количества детей и посчитанное среднее количество выданных кредитов людям с разным количеством детей.

In [28]:
children_set = data.groupby('children')['debt'].sum() / \
    data.groupby('children')['debt'].count() * 100

Выведем полученный результат.

In [29]:
children_set

children
0    7.543822
1    9.234609
2    9.454191
3    8.181818
4    9.756098
5    0.000000
Name: debt, dtype: float64

Посчитаем общее количество выданных кредитов по соотношению к количеству детей. 

In [21]:
grouped_children = data.groupby('children')['debt'].count()
grouped_children

children
0    14149
1     4818
2     2055
3      330
4       41
5        9
Name: debt, dtype: int64

Создадим сводную таблицу.

In [32]:
children_pivot = data.pivot_table(index = ['children'], \
                                columns = ['debt'], \
                                values = 'purpose', aggfunc='count')

children_pivot['ratio'] = children_pivot[1] / children_pivot[0] * 100
children_pivot.iloc[0:3]

debt,0,1,ratio
children,,,
0,13028.0,1063.0,8.159349
1,4364.0,444.0,10.174152
2,1858.0,194.0,10.441335


**Вывод:** 
Заёмщики, не имеющие детей, менее склонны к просрочке по выплатам кредита и более ответсвенные перед обязательствами банка.
Заёмщики с детьми отдают кредиты примерно на одном и том же уровне по сравнению с бездетными заемщиками.
Группа людей у которых больше трёх детей более отвественны, чем семьи с 1-2 детьми, так как им наверняка потребуется хорошая кредитная история для будущих кредитов, например для детей.
Семьи с 1-м и 2-я детьми берут кредит одноразово, поэтому не так сильно ориентируются на срок выплат.
Заёмщиков имеющих больше двух детей не рассматриваем в данном анализе, так как сумма значений менее 2% , что является слишком малым для анализа.

#### 3.2 Есть ли зависимость между семейным положением и возвратом кредита в срок?

Для начала создадим таблицу, куда включим соотношение семейного положения и посчитанное среднее количество выданных кредитов людям с разным семейным положением.

In [30]:
family_status_set = data.groupby('family_status')['debt'].sum() / \
      data.groupby('family_status')['debt'].count() * 100

Выведем полученный результат.

In [31]:
family_status_set

family_status
Не женат / не замужем    9.753483
в разводе                7.064760
вдовец / вдова           6.617647
гражданский брак         9.254808
женат / замужем          7.535360
Name: debt, dtype: float64

Создадим сводную таблицу.

In [33]:
family_status_pivot = data.pivot_table(index=['family_status'],
columns=['debt'], values='total_income', aggfunc='count')
family_status_pivot['ratio'] = family_status_pivot[1] / family_status_pivot[0] * 100
family_status_pivot

debt,0,1,ratio
family_status,,,
Не женат / не замужем,2523,273,10.820452
в разводе,1105,84,7.601810
вдовец / вдова,888,63,7.094595
гражданский брак,3749,385,10.269405
женат / замужем,11334,927,8.178931


**Вывод:**
Семейное положение заёмщиков влияет на возврат кредита в срок.
Исходя из сводной таблицы можно сделать вывод, что самые низкие показатели у разведенных и вдов — можно предположить, что возвращаемость кредитов зависит в том числе от наличия кормильца или супруга в семье.
Следом идут женатые заёмщики - можно предположить, что женатые заёмщики имеют уже детей или планируют заводить их.
Также можно отметить, что свободные заёмщики более ответсвенно относятся срокам возвращения кредитов, что предсказуемо в плане открытия новой кредитной истории.
Заёмщики состоящие в гражданском браке более ответсвенные в плане возврата кредита в определенный срок.


#### 3.3 Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Для начала воспользуемся методом квантиля для подсчета приблизительных средних значений доходов заёмщиков.

In [34]:
#data['total_income'].quantile([0.25,0.5,0.75])

0.25    103000.125541
0.50    145020.801280
0.75    203444.650002
Name: total_income, dtype: float64

In [35]:
pd.qcut(data['total_income'], 4)

0        (195842.0, 2265604.0]
1         (107507.0, 142594.0]
2         (142594.0, 195842.0]
3        (195842.0, 2265604.0]
4         (142594.0, 195842.0]
                 ...          
21520    (195842.0, 2265604.0]
21521     (142594.0, 195842.0]
21522    (20666.999, 107507.0]
21523    (195842.0, 2265604.0]
21524    (20666.999, 107507.0]
Name: total_income, Length: 21331, dtype: category
Categories (4, interval[float64, right]): [(20666.999, 107507.0] < (107507.0, 142594.0] < (142594.0, 195842.0] < (195842.0, 2265604.0]]

С помощью функции income_status распределим уровень доходов на 4 группы и присвоим статусы каждой группе заемщиков.

In [36]:
def income_status(total_income):
    if total_income <= 107620:
            return 'Низкий уровень дохода'
    if total_income <= 142594:
            return 'Средний уровень дохода'
    if total_income < 195818:
            return 'Высокий уровень дохода'
    return 'Сверхвысокий уровень дохода'

Доавим новый столбец income_status в нашу таблицу.

In [37]:
data['income_status'] = data['total_income'].apply(income_status)

Проверим результат вывода нового столбца.

In [38]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category,income_status
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью,Сверхвысокий уровень дохода
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем,Средний уровень дохода
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью,Высокий уровень дохода
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования,Сверхвысокий уровень дохода
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы,Высокий уровень дохода


Создадим таблицу зависимости от статуса дохода и среднего числа кредитов.

In [39]:
income_set = data.groupby('income_status')['debt'].sum() / \
      data.groupby('income_status')['debt'].count() * 100

Отсортируем по возрастанию.

In [40]:
income_set.sort_values()

income_status
Сверхвысокий уровень дохода    7.140180
Низкий уровень дохода          7.996255
Высокий уровень дохода         8.518803
Средний уровень дохода         8.818666
Name: debt, dtype: float64

Создадим сводную таблицу.

In [42]:
income_pivot = data.pivot_table(index=['income_status'], columns=['debt'], values='education_id', aggfunc='count')
income_pivot['ratio'] = income_pivot[1] / income_pivot[0] * 100
income_pivot

debt,0,1,ratio
income_status,,,
Высокий уровень дохода,4768,444,9.312081
Низкий уровень дохода,4913,427,8.691227
Сверхвысокий уровень дохода,4955,381,7.689203
Средний уровень дохода,4963,480,9.671570


Проверим количество людей в каждой из групп.

In [43]:
data['income_status'].value_counts(normalize = True)

Средний уровень дохода         0.255169
Низкий уровень дохода          0.250340
Сверхвысокий уровень дохода    0.250152
Высокий уровень дохода         0.244339
Name: income_status, dtype: float64

**Вывод:** 
Есть незначительная разница в возвращаемости кредитов от уровня доходов.
Хуже всего показатель у людей со сверхвысокими и низкими доходами. Если зависимость возвращаемости кредитов у людей с низкими доходами очевидна, то сверхвысокие объяснить не могу. С количеством человек в каждой из групп — порядок.

#### 3.4 Как разные цели кредита влияют на его возврат в срок?

Для начала составим словарь из двух списков.

In [57]:
credit_purpose_keys= []
credit_purpose_list = list(data.purpose.unique())
for value in range(len(credit_purpose_list)):
    if 'образован' in credit_purpose_list[value]:
        credit_purpose_keys.append('Образование')
    elif 'авто' in credit_purpose_list[value]:
        credit_purpose_keys.append('Автомобиль')
    elif 'свадь' in credit_purpose_list[value]:
        credit_purpose_keys.append('Свадьба')
    else:
        credit_purpose_keys.append('Недвижимость')
        
credit_purpose_keys

['Недвижимость',
 'Автомобиль',
 'Образование',
 'Свадьба',
 'Недвижимость',
 'Образование',
 'Свадьба',
 'Недвижимость',
 'Недвижимость',
 'Недвижимость',
 'Недвижимость',
 'Недвижимость',
 'Недвижимость',
 'Недвижимость',
 'Автомобиль',
 'Автомобиль',
 'Недвижимость',
 'Недвижимость',
 'Недвижимость',
 'Недвижимость',
 'Автомобиль',
 'Образование',
 'Автомобиль',
 'Образование',
 'Автомобиль',
 'Свадьба',
 'Образование',
 'Недвижимость',
 'Недвижимость',
 'Образование',
 'Автомобиль',
 'Автомобиль',
 'Образование',
 'Образование',
 'Недвижимость',
 'Автомобиль',
 'Недвижимость',
 'Образование']

Проверим список полученных значений.

In [58]:
credit_purpose_list

['покупка жилья',
 'приобретение автомобиля',
 'дополнительное образование',
 'сыграть свадьбу',
 'операции с жильем',
 'образование',
 'на проведение свадьбы',
 'покупка жилья для семьи',
 'покупка недвижимости',
 'покупка коммерческой недвижимости',
 'покупка жилой недвижимости',
 'строительство собственной недвижимости',
 'недвижимость',
 'строительство недвижимости',
 'на покупку подержанного автомобиля',
 'на покупку своего автомобиля',
 'операции с коммерческой недвижимостью',
 'строительство жилой недвижимости',
 'жилье',
 'операции со своей недвижимостью',
 'автомобили',
 'заняться образованием',
 'сделка с подержанным автомобилем',
 'получение образования',
 'автомобиль',
 'свадьба',
 'получение дополнительного образования',
 'покупка своего жилья',
 'операции с недвижимостью',
 'получение высшего образования',
 'свой автомобиль',
 'сделка с автомобилем',
 'профильное образование',
 'высшее образование',
 'покупка жилья для сдачи',
 'на покупку автомобиля',
 'ремонт жилью',
 '

Создаем словарь, пользуясь методом zip.

In [59]:
credit_purpose_dict = dict(zip(credit_purpose_list, credit_purpose_keys))

Добавим новый столбец credit_purpose_status к нашей таблице

In [60]:
data['credit_purpose_status'] = data['purpose'].map(credit_purpose_dict)

Создадим сводную таблицу.

In [64]:
credit_purpose_pivot = data.pivot_table(index=['credit_purpose_status'], columns=['debt'], values='education_id', aggfunc='count')
credit_purpose_pivot['ratio'] = credit_purpose_pivot[1] / credit_purpose_pivot[0] * 100
credit_purpose_pivot

debt,0,1,ratio
credit_purpose_status,,,
Автомобиль,3879,400,10.311936
Недвижимость,9971,780,7.822686
Образование,3619,369,10.196187
Свадьба,2130,183,8.591549


**Вывод:** 
    Цели кредита влияют на его возврат в срок.
Заемщики, берущие кредит для приобретения/проведение операций с жильем, наиболее ответственны и менее склонны нарушать обязательства по выплатам кредита в срок.
Худший показатель у заёмщиков, которые берут кредит на покупку автомобиля. 

#### 3.5 Приведите возможные причины появления пропусков в исходных данных.

**Ответ:** 
В данной таблице данных мы выявили пропуски в исходных данных в двух столбцах:
- days_employed — общий трудовой стаж в днях;
- total_income — ежемесячный доход.

Возможных причин появления таких пропусков в исходных данных несколько:
- неправильный ввод данных;
- сокрытие информации;
- мошенничество или обман;
- технический сбой при вводе, записи, перезаписи данных, а также при переносе с одного устройства на другое.

#### 3.6 Объясните, почему заполнить пропуски медианным значением — лучшее решение для количественных переменных.

**Ответ:** 
Медиана показывает центральное значение в выборке, если наблюдений нечетное количество и среднее арифметическое двух значений, если количество наблюдений в выборке четно. Оценка среднего значения выборки — зачастую важна во многих исследовательских вопросах. 

В контексте бизнеса разница между средним арифметическим и медианой также важна, так как использование неверной метрики может серьезно сказаться на результатах проведения акции или затруднить достижение цели.

Если сравнивать среднее значение и медиану. Среднее значение заполненных пропусках в большинстве случаев будут очень сильно влиять на результат данных.

Медиана более точно характеризует данные с несимметричным распределением или с любым распределением, не
соответствующим нормальному закону.

### Шаг 4: общий вывод.

***Заказчик — кредитный отдел банка.***

***Задача: Влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок.*** 

Основываясь на полученных данных можно прийти к следующему выводу:
Семейное положение и наличие детей в семье безусловно влияют на факт погашения кредита в указанный срок.
Также мы можем увидеть, что вероятность клиента принадлежащего к группе "многодетные" стать должником выше, а вероятность клиента принадлежащего к группе "не женат / не замужем" стать должником ниже. 
Поэтому логично будет предположить, что наиболее безопасными для банка заемщиками являются семьи без детей и люди являющиеся вдовой/вдовцом.

- Чаще всего целью кредита являются: недвижимость/жилье, автомобиль и образование;
- Цели, которые чаще других не выплачиваются вовремя являются: недвижимость и автомобиль;
- Заёмщики без детей чаще других не выплачивали кредит вовремя;
- Клиенты, которые были женаты/замужем чаще других не выплачивали кредит вовремя;

Общий результат: на факт погашения кредита в срок влияет почти каждый фактор, начиная от количества детей, заканчивая целью кредита. 

Есть ли зависимость между количеством детей и возвратом кредита в срок?

- Семьи у которых нет детей - 8.159349 %
- Семьи у которых 1 ребенок - 10.174152 %
- Семьи у которых 2 ребенка - 10.441335 %

Разница в процентном соотношении возврата кредита в срок у семей, имеющих 2 ребенка по отношению к семьям у которых нет детей составляет ***2,28 %***. 

Есть ли зависимость между семейным положением и возвратом кредита в срок?

- Вдовец/вдова        - 7.094595 %
- В разводе	        - 7.601810 %
- Женат/замужем       - 8.178931 %
- Гражданский брак    - 10.269405 %
- Не женат/не замужем	- 10.820452 %

Разница в процентном соотношении возврата кредита в срок у заёмщиков в категории вдовец/вдова по отношению к заемщикам в категории не женат/не замужем составляет ***3,72 %***.

Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

- Сверхвысокий уровень дохода - 7.689203 %
- Низкий уровень дохода	    - 8.691227 %
- Высокий уровень дохода	    - 9.312081 %
- Средний уровень дохода	    - 9.671570 %

Разница в процентном соотношении возврата кредита в срок у заёмщиков со сверхвысоким уровнем дохода по отношению к заёмщикам со средним уровнем дохода состаляет ***1,92 %***.

Как разные цели кредита влияют на его возврат в срок?

- Недвижимость - 7.800000 %
- Свадьба	     - 8.495822 %
- Образование	 - 10.170893 %
- Автомобиль	 - 10.288066 %

Разница в процентном соотношении возврата кредита в срок у заёмщиков, берущих кредит на недвижимость по отношению к заёмщикам, берущим кредит на автомобиль составляет ***2,48 %***.

***Рекомендации для банка:***

 - Заёмщики, у которых есть дети - наиболее отвественные в плане возврата кредита в срок;
 - Идеальными заёмщиками могут быть свободные клиенты и клиенты, живущие в гражданском браке;
 - Как ни странно заёмщики со средним доходом будут являться приоритетными клиентами для банка;
 - Заёмщики, берущие кредиты на автомобиль и образование более привлекательны для банка.